# <span style="font-family:Courier New; color:#CCCCCC">**Named Entity Recognition CRF**</span>

## <span style="font-family:Courier New; color:#336666">**Load Data and Imports**</span>

In [1]:
from preprocessing import convert_BIO
from NER_evaluation import *
from feature_getter import Feature_getter
import pycrfsuite

import nltk
nltk.download('conll2002')
from nltk.corpus import conll2002

esp_train = conll2002.iob_sents('esp.train') 
esp_val = conll2002.iob_sents('esp.testa')
esp_test = conll2002.iob_sents('esp.testb')

ned_train = conll2002.iob_sents('ned.train')
ned_val = conll2002.iob_sents('ned.testa')
ned_test = conll2002.iob_sents('ned.testb')

[nltk_data] Downloading package conll2002 to
[nltk_data]     C:\Users\Jordi\AppData\Roaming\nltk_data...
[nltk_data]   Package conll2002 is already up-to-date!


## <span style="font-family:Courier New; color:#336666">**Train Classifier**</span>

In [2]:
esp_train = convert_BIO(esp_train)
model = nltk.tag.CRFTagger(feature_func = Feature_getter())
model.train(esp_train, 'model.crf.tagger')

In [ ]:
esp_test = convert_BIO(esp_test)
X_esp_test = [[word[0] for word in sent] for sent in esp_test]
pred = model.tag_sents(X_esp_test)

In [4]:
results, results_agg_ent = compute_metrics(esp_test, pred)
results

{'correct': 2577,
 'incorrect': 530,
 'partial': 108,
 'missed': 393,
 'spurious': 268,
 'possible': 3608,
 'actual': 3483,
 'precision': 0.739879414298019,
 'recall': 0.7142461197339246,
 'F1-score': 0.726836835425187}

## <span style="font-family:Courier New; color:#336666">**Feature selection**</span>

In this section we will try to perform the feature selection in order to achieve the best performance we can. We first contemplate the isolated effects of these features (only activating these).

In [2]:
train_sents = convert_BIO(esp_train)
test_sents = convert_BIO(esp_test)
val_sents = convert_BIO(esp_val)

X_val_sents = [[word[0] for word in sent] for sent in val_sents]
X_test_sents = [[word[0] for word in sent] for sent in test_sents]

### Best n-gram

Lets start by finding how the introduction of bigrams and trigrams can affect the model.

**Unigram**

In [5]:
model = nltk.tag.CRFTagger()
model.train(train_sents, 'model.crf.tagger')

pred = model.tag_sents(X_val_sents)
results, _ = compute_metrics(val_sents, pred)

res_uni = results['F1-score']

In [6]:
res_uni

0.6810721454717203

**Bigram**

In [5]:
model = nltk.tag.CRFTagger(feature_func = Feature_getter(bigram = True, trigram = False, morphology = False, length = False, prefix = False,
                 sufix = True, lemma = False, POS = False, shape = False))
model.train(train_sents, 'model.crf.tagger')

pred = model.tag_sents(X_val_sents)
results, _ = compute_metrics(val_sents, pred)

res_bi = results['F1-score']

In [6]:
res_bi

0.6605460832240371

**Trigram**

In [8]:
model = nltk.tag.CRFTagger(feature_func = Feature_getter(bigram = True, trigram = True, morphology = False, length = False, prefix = False,
                 sufix = True, lemma = False, POS = False, shape = False))
model.train(train_sents, 'model.crf.tagger')

pred = model.tag_sents(X_val_sents)
results, _ = compute_metrics(val_sents, pred)

res_tri = results['F1-score']

In [9]:
res_tri

0.6542188607894115

As we can see, the performance has rather decreased introducing these features. It could be due to the extra dimensionality

### Including morphology

**Without morphology**

In [12]:
res_uni 

0.6810721454717203

**Including morphology**

In [14]:
model = nltk.tag.CRFTagger(feature_func = Feature_getter(bigram = False, trigram = False, morphology = True, length = False, prefix = False,
                 sufix = True, lemma = False, POS = False, shape = False))
model.train(train_sents, 'model.crf.tagger')

pred = model.tag_sents(X_val_sents)
results, _ = compute_metrics(val_sents, pred)

best_morphology = results['F1-score']

In [15]:
best_morphology

0.6142925890279114

Somehow we can see a drastic performance decrease when introducing morphology alone

### Including all other variables


We will continue with unigrams, but try to see the performance when incloduing all variables of the token we are analising at the moment

In [17]:
model = nltk.tag.CRFTagger(feature_func = Feature_getter(bigram = False, trigram = False, morphology = True, length = True, prefix = True,
                 sufix = True, lemma = True, POS = True, shape = True))
model.train(train_sents, 'model.crf.tagger')

pred = model.tag_sents(X_val_sents)
results, _ = compute_metrics(val_sents, pred)

best_other = results['F1-score']

In [18]:
best_other

0.6606003308910423

**Interactions(all)**

Now we try to introduce bigrams and trigrams

In [20]:
model = nltk.tag.CRFTagger(feature_func = Feature_getter())
model.train(train_sents, 'model.crf.tagger')

pred = model.tag_sents(X_val_sents)
results, _ = compute_metrics(val_sents, pred)

best_all = results['F1-score']

In [21]:
best_all

0.682744960969358

We get more or less the same performance as we get without the custom feature extractor

## <span style="font-family:Courier New; color:#336666">**Hiperparameters selection**</span>

We will begin with hiperparameters selection. However, we will perform it on the base features of the classifier. The reason lies in the runtime that a training with all features bears with it, along with the assumption that the hiperparameters doesnt have distinct interactions among the different features

We will try to do a custom function that does a gridsearch over the different values we try to test.

In [22]:
import pandas as pd

In [23]:
hyperparameters = {
    'c1': [0.1, 0.5, 1.0],
    'c2': [0.1, 0.5, 1.0],
    'max_iterations': [50, 100, 200]
}

In [31]:
def gridsearch_cv(hyperparameters,train_sents,val_sents,X_val_sents):
    results_df = pd.DataFrame(columns = ['c1', 'c2', 'max_iterations', 'F1-score'])
    best_f1 = 0
    best_params = {}
    num_combinations = len(hyperparameters['c1']) * len(hyperparameters['c2']) * len(hyperparameters['max_iterations'])
    current_combination = 0
    for c1 in hyperparameters['c1']:
        for c2 in hyperparameters['c2']:
            for max_iter in hyperparameters['max_iterations']:
                current_combination += 1
                print(f'Fitting model {current_combination} of {num_combinations}', end = '\r')
                model = nltk.tag.CRFTagger(training_opt = {'c1': c1, 'c2': c2, 'max_iterations': max_iter})
                model.train(train_sents, 'model.crf.tagger')

                pred = model.tag_sents(X_val_sents)
                results, _ = compute_metrics(val_sents, pred)
                results_df.loc[len(results_df)] = [c1, c2, max_iter, results['F1-score']]
                if results['F1-score'] > best_f1:
                    best_f1 = results['F1-score']
                    best_params = {'c1': c1, 'c2': c2, 'max_iterations': max_iter}

    return best_f1,best_params,results_df

In [32]:
best, best_params,dataframe = gridsearch_cv(hyperparameters,train_sents,val_sents,X_val_sents)

In [43]:
dataframe.sort_values(by = 'F1-score', ascending = False).head(10)


,c1,c2,max_iterations,F1-score
1,0.1,0.1,100.0,0.706310
2,0.1,0.1,200.0,0.704532
0,0.1,0.1,50.0,0.695958
10,0.5,0.1,100.0,0.694897
11,0.5,0.1,200.0,0.693380
9,0.5,0.1,50.0,0.690627
5,0.1,0.5,200.0,0.689566
4,0.1,0.5,100.0,0.688490
3,0.1,0.5,50.0,0.684173
14,0.5,0.5,200.0,0.680696


 Now lets try with the complete model

In [36]:
def gridsearch_cv_complete(hyperparameters,train_sents,val_sents,X_val_sents):
    results_df = pd.DataFrame(columns = ['c1', 'c2', 'max_iterations', 'F1-score'])
    best_f1 = 0
    best_params = {}
    num_combinations = len(hyperparameters['c1']) * len(hyperparameters['c2']) * len(hyperparameters['max_iterations'])
    current_combination = 0
    for c1 in hyperparameters['c1']:
        for c2 in hyperparameters['c2']:
            for max_iter in hyperparameters['max_iterations']:
                current_combination += 1
                print(f'Fitting model {current_combination} of {num_combinations}', end = '\r')
                model = nltk.tag.CRFTagger(training_opt = {'c1': c1, 'c2': c2, 'max_iterations': max_iter}, feature_func = Feature_getter())
                model.train(train_sents, 'model.crf.tagger')

                pred = model.tag_sents(X_val_sents)
                results, _ = compute_metrics(val_sents, pred)
                results_df.loc[len(results_df)] = [c1, c2, max_iter, results['F1-score']]
                if results['F1-score'] > best_f1:
                    best_f1 = results['F1-score']
                    best_params = {'c1': c1, 'c2': c2, 'max_iterations': max_iter}

    return best_f1,best_params,results_df

In [38]:
best_complete, best_params_complete,dataframe_complete = gridsearch_cv_complete(hyperparameters,train_sents,val_sents,X_val_sents)

In [42]:
dataframe_complete.sort_values(by = 'F1-score', ascending = False).head(10)

,c1,c2,max_iterations,F1-score
8,0.1,1.0,200.0,0.680509
7,0.1,1.0,100.0,0.680135
17,0.5,1.0,200.0,0.677962
5,0.1,0.5,200.0,0.677239
14,0.5,0.5,200.0,0.676632
13,0.5,0.5,100.0,0.675064
4,0.1,0.5,100.0,0.673350
6,0.1,1.0,50.0,0.672271
2,0.1,0.1,200.0,0.671928
11,0.5,0.1,200.0,0.671798


In [44]:
hyperparameters_default = {'c1':0.1, 'c2':0.1, 'max_iterations':100}
hyperparameters_custom = {'c1':0.1, 'c2':1, 'max_iterations':100}

Lets try to play with a parameter that can help the score of custom features addition:             
- 'feature.minfreq': The minimum frequency of features.
- 'feature.possible_states': Force to generate possible state features.
-  'feature.possible_transitions': Force to generate possible transition features.

In [8]:
model = nltk.tag.CRFTagger(feature_func = Feature_getter(), training_opt = {
    'c1': 0.1,   # coefficient for L1 penalty
    'c2':1.0,  # coefficient for L2 penalty
    'max_iterations': 100,  # stop earlier

    # include transitions that are possible, but not observed
    'feature.possible_transitions': True
})
model.train(train_sents, 'model.crf.tagger')

pred = model.tag_sents(X_val_sents)
results, _ = compute_metrics(val_sents, pred)

res = results['F1-score']

In [9]:
res

0.6805830903790087

We improve only a little bit.

Let's try now with feature possible states

In [10]:
model = nltk.tag.CRFTagger(feature_func = Feature_getter(), training_opt = {
    'c1': 0.1,   # coefficient for L1 penalty
    'c2':1.0,  # coefficient for L2 penalty
    'max_iterations': 100,  # stop earlier

    
    'feature.possible_states': True
})
model.train(train_sents, 'model.crf.tagger')

pred = model.tag_sents(X_val_sents)
results, _ = compute_metrics(val_sents, pred)

res = results['F1-score']

In [11]:
res

0.684733091928513

We see a little more improvement here

Finally, lets try to establish minfreq.

In [12]:
model = nltk.tag.CRFTagger(feature_func = Feature_getter(), training_opt = {
    'c1': 0.1,   # coefficient for L1 penalty
    'c2':1.0,  # coefficient for L2 penalty
    'max_iterations': 100,  # stop earlier

    
    'feature.minfreq': 5
})
model.train(train_sents, 'model.crf.tagger')

pred = model.tag_sents(X_val_sents)
results, _ = compute_metrics(val_sents, pred)

res = results['F1-score']

In [13]:
res

0.659353483486988

In [3]:
model = nltk.tag.CRFTagger(feature_func = Feature_getter(), training_opt = {
    'c1': 0.1,   # coefficient for L1 penalty
    'c2':1.0,  # coefficient for L2 penalty
    'max_iterations': 100,  # stop earlier
    'feature.possible_states': True,
    'feature.possible_transitions': True
})
model.train(train_sents, 'model.crf.tagger')

pred = model.tag_sents(X_val_sents)
results, _ = compute_metrics(val_sents, pred)

res = results['F1-score']

In [4]:
res

0.683313885647608

We cannot see much improvement with the introduction of these two variables